# Multi-armed Bandits Algorithms

This notebook explains and reproduces the most popular multi-armed bandits (MAB) algorithms:
- epsilon-greedy
- UCB
- Thompson Sampling

### What?
MAB is a classic reinforcement learning problem for balancing the trade-off between exploration vs exploitation.

Essentially, MAB formulates various kinds of sequential decision-making problems; thus, it is applicable to a wide range of scenarios.

### Example
In a casino, a gambler plays a slot machine that has `K` pulling arms (the story if there are `K` slot machines with one arm). 

How to leave the casino without being bankrupt? Or even with some earnings?

##### Rules
1. Each arm has a reward distribution that is initially unknown to the gambler.
2. Each round, the gambler can only play one arm and only observe the reward of that very arm.

##### Goal
Maximize the cumulative reward before he bankrupts or the casino closes.

# Libraries

In [29]:
import numpy as np
import matplotlib.pyplot as plt

# Problem (Bandit Environment)

Don't get confused.

MAB algorithms solves MAB problems.

We need to define a problem first before investigating how each algorithms perform.

In [30]:
class AbstractMAPProblem(object):
    """
    The basics of a generic MAB problem
    """
    def __init__(self, name: str, num_arms: int):
        self.bandit_name = name
        self.num_arms = num_arms

    def get_reward(self, arm_choice: int):
        raise NotImplementedError

### The gambler, Jack
In front of Jack, the gambler, is a fancy slot machine, called the Fruit Machine.
```
  ______          _ _     __  __            _     _            
 |  ____|        (_) |   |  \/  |          | |   (_)           
 | |__ _ __ _   _ _| |_  | \  / | __ _  ___| |__  _ _ __   ___ 
 |  __| '__| | | | | __| | |\/| |/ _` |/ __| '_ \| | '_ \ / _ \
 | |  | |  | |_| | | |_  | |  | | (_| | (__| | | | | | | |  __/
 |_|  |_|   \__,_|_|\__| |_|  |_|\__,_|\___|_| |_|_|_| |_|\___|
```

##### Fruit Machine
There are 3 arms/buttons that correspond to 3 different fruits (cherry, orange, and manage).

Each time Jack pays $1 to play.

This machine only requires one single brain cell to play:
1. choose a fruit & pull its arm -> the wheel starts to spin -> the wheel stops
2. win the prize (get back $2) or lose ($1 is gone)
3. play again (going home is never an option for Jack :)

In [31]:
class FruitSlotMachine(AbstractMAPProblem):
    """
    An Bernoulli Bandit. Reward obeys a Bernoulli distribution
    This machine has 3 arms:
    0 -- cherry
    1 -- orange
    2 -- mango
    """
    def __init__(self, distribution: list = None):
        """
        this machine supports two factory modes:
        1. set a fixed win rate for each arm
        2. randomize the win rate for each arm
        """
        super().__init__("Fruit Machine", 3)
        if distribution is not None: # mode 1
            assert len(distribution) == self.num_arms
            self.distribution = distribution
        else: # mode 2
            self.distribution = [np.random.random() for _ in range(self.num_arms)]
        
        self.best_cumulative_reward = max(self.distribution)

    def get_reward(self, arm_choice: int):
        """
        return the sampled reward for the given arm choice
        
        Output:
        reward -- 0 or 1
        """
        if np.random.random() < self.distribution[arm_choice]:
            return 1
        else:
            return 0

As we are the owner of the Fruit Machine, we can adjust the win rate:
- cherry -> 30% win, 70% lose
- orange -> 50% win, 50% lose
- manago -> 70% win, 30% lose

Obviously, we know betting on manago is always the best.
Gamblers need to play tons of times before they eventually realize this secret.

In [32]:
# initialize Fruit Machine with a fixed win rate / distribution
FruitMachine = FruitSlotMachine([0.3, 0.5, 0.7])
Fruits = ['cherry', 'orange', 'mango']

# Algorithms
An MAB algorithm solves MAB problems.
### What does it do?
Essentially, an MAB algorithm is a strategy (aka, policy) on how to perform the next move, such that we gain better understanding of the problem and the hidden distributions.
The algorithm will:
1. `choose_arm`: make a decision on actions (which fruit to bet on?)
2. observe the result and `update_belief` based on the arm choice and result/reward.
3. repeat for some iterations


In [33]:
class AbstractMABAlgorithm:
    def __init__(self, MABproblem: FruitSlotMachine) -> None:
        self.MABproblem = MABproblem
        self.N_a = np.array([0] * self.MABproblem.num_arms) # the number of choice on arm a
        self.regret = 0 # cumulative regret
        self.regret_history = [0] # regret history at each step
    
    def choose_arm(self) -> int:
        """choose an arm/action"""
        raise NotImplementedError

    def update_belief(self, arm_choice, reward) -> None:
        """update the model"""
        raise NotImplementedError
    
    def update_regret(self, arm_choice: int):
        self.regret += self.MABproblem.best_cumulative_reward - self.MABproblem.distribution[arm_choice]
        self.regret_history.append(self.regret)

    def run(self, num_iterations: int):
        for self.iteration in range(num_iterations):
            # choose an arm & observe reward
            arm_choice = self.choose_arm()
            reward = self.MABproblem.get_reward(arm_choice)
            # update model & regret
            self.N_a[arm_choice] += 1
            self.update_belief(arm_choice, reward)
            self.update_regret(arm_choice)

## Stocastic Bandits
In the simplest form, stochastic bandits deal with IID reward model. The independent and identical distributed reward D (with an expected value μ and a variance σ) is initially hidden to the agent. At each time step t ∈ T, an agent selects one of the K arms and only observe the reward from the selected arm a, known as bandit feedback. Reward value is simplified to be 0 or 1 (or within [0, 1]). It is the goal of an agent to realize the reward distributions and maximizes the cumulative reward in T iterations.
To access a bandit algorithm, we need to define Regret. Given IID rewards, the highest cumulative reward can be obtained by repeatedly choosing the optimal arm for T iterations.

### Uniform Exploration
The simplest strategies.

Uniform Exploration ignores any observed rewards and explores each arm uniformly.
For example, Jacky with 1000$ plays each fruit for 333 times, no matter of winning or losing.

Notice Uniform Exploration is known as Explore-first algorithm. After exploration phase, simply exploit the best arm thereafter.

In [34]:
class UniformExploration(AbstractMABAlgorithm):
    def __init__(self, MABproblem: FruitSlotMachine, initial_probability: float=1.0) -> None:
        super().__init__(MABproblem)
        self.probabilities = np.array([initial_probability] * self.MABproblem.num_arms)

    def choose_arm(self) -> int:
        arm = np.argmin(self.N_a)
        return arm

    def update_belief(self, arm_choice, reward) -> None:
        self.probabilities[arm_choice] += (reward - self.probabilities[arm_choice]) / (self.N_a[arm_choice]) 

##### Performance
Jack has to make a decision on how to divide the $1000 chips.
Here, Jack decides to use $999 to explore and $1 to exploit.

In [35]:
uniform_explore = UniformExploration(FruitMachine)
uniform_explore.run(num_iterations=999)
print(uniform_explore.probabilities)
for i, winrate in enumerate(uniform_explore.probabilities):
    print(f"Approximate winrate for {Fruits[i]}:", round(winrate, 3))
    print(f"\t#times played = {uniform_explore.N_a[i]}")

[0.33933934 0.51351351 0.69069069]
Approximate winrate for cherry: 0.339
	#times played = 333
Approximate winrate for orange: 0.514
	#times played = 333
Approximate winrate for mango: 0.691
	#times played = 333


##### Questions
1. What if Jack use $500 to explore and $500 to exploit?
2. What if too less exploration?

### Epsilon-greedy
Explores a random arm at a probability of ε, and exploit the current best arm otherwise, i.e. greedy choice.

##### Epsilon value
Cesa-Bianchi and Fischer (1998) proved that the regret has logarithmic bound if ε is proportionally to the reciprocal of iterations (i.e. ε ∝ 1t ), though a widely-cited empirical evaluation was not able to prove the practicality of this theoretically sound statement (Vermorel and Mohri, 2005)

We use fixed a epsilon value.

In [36]:
class EpsilonGreedy(AbstractMABAlgorithm):
    def __init__(self, MABproblem: FruitSlotMachine, epsilon: float, initial_probability: float=1.0) -> None:
        super().__init__(MABproblem)
        assert epsilon is None or 0 <= epsilon <= 1.0
        self.epsilon = epsilon
        self.probabilities = np.array([initial_probability] * self.MABproblem.num_arms)

    def choose_arm(self) -> int:
        if np.random.random() < self.epsilon:
            arm = np.random.randint(0, self.MABproblem.num_arms)
        else:
            arm = self.probabilities.argmax()
        return arm

    def update_belief(self, arm_choice, reward) -> None:
        self.probabilities[arm_choice] += (reward - self.probabilities[arm_choice]) / (self.N_a[arm_choice]) 

##### Performance
Jack, the gambler, plays for a 1000 times following the Epsilon Greedy algorithm and he keeps track of the winrate of each fruit/arm/action.

Jack chooses to evenly splite his exploration and exploitation (50% - 50%. Approaximately 500 times for choosing randomly)

Thus, `epsilon = 0.5`

Jack uses epsilon greedy to successfully approximates the winrate for each fruit.

In [37]:
epsilon_greedy = EpsilonGreedy(FruitMachine, epsilon=0.5)
epsilon_greedy.run(num_iterations=1000)
print(epsilon_greedy.probabilities)
for i, winrate in enumerate(epsilon_greedy.probabilities):
    print(f"Approximate winrate for {Fruits[i]}:", round(winrate, 3))
    print(f"\t#times played = {epsilon_greedy.N_a[i]}")


[0.30718954 0.47701149 0.7013373 ]
Approximate winrate for cherry: 0.307
	#times played = 153
Approximate winrate for orange: 0.477
	#times played = 174
Approximate winrate for mango: 0.701
	#times played = 673


##### Questions
1. How does `epsilon` affect the result? Is larger or smaller `epsilon` works better?
2. How does `# iterations` affect the result? In other word, how does the cumulative regret change over time?

### Upper Confidence Bounds (UCB)
The basic algorithm, UCB1, tracks the number of trails n on each arm besides the expected value μ. The agent selects an arm a based on the optimistic estimates (i.e. maximum of UCB):

Repetitively selecting the arm that maximizes UCB will eventually converge to the best arm a∗, which is easily proved by contradiction of UCBt(at) ≥ UCBt(a∗).

In [38]:
class UCB(AbstractMABAlgorithm):
    def __init__(self, MABproblem: FruitSlotMachine, initial_probability: float=1.0) -> None:
        super().__init__(MABproblem)
        self.probabilities = np.array([initial_probability] * self.MABproblem.num_arms)

    def choose_arm(self) -> int:
        t = self.iteration+1
        UCB = self.probabilities + np.sqrt(2 * np.log(t) / (self.N_a + 1))
        arm = np.argmax(UCB)
        return arm

    def update_belief(self, arm_choice, reward) -> None:
        self.probabilities[arm_choice] += (reward - self.probabilities[arm_choice]) / (self.N_a[arm_choice]) 

##### Performance
UCB can also help Jack successfully approximates the winrate for each fruit.


In [39]:
ucb = UCB(FruitMachine)
ucb.run(num_iterations=1000)
print(ucb.probabilities)
for i, winrate in enumerate(ucb.probabilities):
    print(f"Approximate winrate for {Fruits[i]}:", round(winrate, 3))
    print(f"\t#times played = {ucb.N_a[i]}")


[0.26530612 0.4822695  0.6654321 ]
Approximate winrate for cherry: 0.265
	#times played = 49
Approximate winrate for orange: 0.482
	#times played = 141
Approximate winrate for mango: 0.665
	#times played = 810


## Bayesian Bandits
Previous algorithms assume the agent has no knowledge of the reward distribution; therefore, we can only make general but conservative estimations of the bound by Hoeffding’s Inequality. How would an agent improve if the prior distribution is provided?

The Bayesian bandits leverage main concepts of Bayesian statistics to solve stochastic bandit problems.

### Bayesian UCB
Jack studies some distribution model and expects the mean reward of each fruit follows Gaussian distribution.

In [40]:
class BayesianUCB(AbstractMABAlgorithm):
    def __init__(self, MABproblem: FruitSlotMachine, sigma=3, initial_a=1., initial_b=1.) -> None:
        super().__init__(MABproblem)
        self.sigma = sigma
        self.a = np.array([initial_a] * MABproblem.num_arms)
        self.b = np.array([initial_b] * MABproblem.num_arms)

    def getBetaStdDev(self) -> float:
        variance = np.multiply(self.a, self.b) / np.multiply(np.square(self.a + self.b), (self.a + self.b + 1))
        return np.sqrt(variance)

    def choose_arm(self) -> int:
        UCB = self.a / (self.a + self.b) + self.getBetaStdDev() * self.sigma
        arm = np.argmax(UCB)
        return arm

    def update_belief(self, arm_choice, reward) -> None:
        self.a[arm_choice] += reward
        self.b[arm_choice] += 1 - reward

##### Performance

In [41]:
bayes_ucb = BayesianUCB(FruitMachine)
bayes_ucb.run(num_iterations=1000)
print(bayes_ucb.a, bayes_ucb.b)

[  4.   7. 678.] [  8.  11. 298.]


### Thompson Sampling

Formally, each arm a has a Bernoulli distribution (i.e. reward received is either a success or a failure), with the mean denoted by μ; it is common to model the mean μ of each arm using Beta distribution. Each arm corresponds to a random variable θ ∈ [0, 1] that represents the probability of receiving a success. The agent starts off with a given belief of the distribution (e.g. αk = 1, βk = 1 indicating the initial success rate of arm a is uniform); at each iteration t, the agent chooses an optimal arm according to the current prior probability, and the belief is updated by posterior distribution upon receiving the reward

In [50]:
class ThompsonSamping(AbstractMABAlgorithm):
    def __init__(self, MABproblem: FruitSlotMachine, initial_a=1, initial_b=1) -> None:
        super().__init__(MABproblem)
        self.a = np.array([initial_a] * MABproblem.num_arms)
        self.b = np.array([initial_b] * MABproblem.num_arms)
    
    def choose_arm(self) -> int:
        samples = [0] * self.MABproblem.num_arms
        for i in range(self.MABproblem.num_arms):
            samples[i] = np.random.beta(self.a, self.b).sum()
        arm = np.argmax(samples)
        return arm
    
    def update_belief(self, arm_choice, reward) -> None:
        self.a[arm_choice] += reward
        self.b[arm_choice] += 1 - reward

##### Performance

In [52]:
thompson_sampling = ThompsonSamping(FruitMachine)
thompson_sampling.run(num_iterations=1000)
print(thompson_sampling.a, thompson_sampling.b)

[101 176 224] [243 164  98]
